This demo uses the following implemented stories:
- RSPY-25
- RSPY-85
- RSPY-100
- RSPY-115
- RSPY-139

## Configuration

In [12]:
# Set local or cluster configuration
import os

if os.getenv("RSPY_LOCAL_MODE") == "1":
    url_catalog = "http://rs-server-catalog:8000"
    HEADERS={}
    local_mode = True
else:
    url_catalog = os.environ["RSPY_WEBSITE"]
    HEADERS={"headers": {"x-api-key": os.environ["RSPY_APIKEY"]}}
    local_mode = False

print(f"Using: {RS_SERVER_ROOT_URL}")

import requests
import json

Using: https://dev-rspy.esa-copernicus.eu


Install the needed libraries

In [5]:
!pip install boto3
if local_mode:
    !(cd $RSPY_WHL_DIR && pip install rs_client_libraries-*.whl )


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [7]:
# We'll use boto3 to monitor the s3 bucket. 
# Note: the S3_ACCESSKEY, S3_SECRETKEY and S3_ENDPOINT are given in the docker-compose.yml file.
import boto3
import os

s3_session = boto3.session.Session()
s3_client = s3_session.client(
    service_name="s3",
    aws_access_key_id=os.environ["S3_ACCESSKEY"],
    aws_secret_access_key=os.environ["S3_SECRETKEY"],
    endpoint_url=os.environ["S3_ENDPOINT"],
    region_name=os.environ["S3_REGION"],
)

buckets = ["rs-cluster-temp", "rs-cluster-catalog"]
bucket_dir = "stations"
bucket_url = f"s3://{buckets[0]}/{bucket_dir}"

# Only in local mode
if local_mode:
    
    # If bucket is already created, clear all files in order to start fresh for each demo. 
    for b in buckets:
        if b in [bucket["Name"] for bucket in s3_client.list_buckets()["Buckets"]]:
            if 'Contents' in s3_client.list_objects(Bucket=b):
                objects = s3_client.list_objects(Bucket=b)['Contents']
                for obj in objects:
                    # clear up the bucket
                    s3_client.delete_object(Bucket=b, Key=obj['Key'])
        else:
            s3_client.create_bucket(Bucket=b)
    for b in buckets:
        print(f"Is {b} empty ?: ", 'Contents' not in s3_client.list_objects(Bucket=b))
    
    # Truncate the items table from catalog also if this is not the first run !
    #docker exec -it catalog-db psql -U postgres -d catalog -c "TRUNCATE items"

A bucket "tmp-download" is created for this demo. Thus, the cadip and adgs prefect flows will be asking for the rs-server endpoints to download the files from CADIP and ADGS stations and to upload them to "s3://tmp-download/stations/".
After a successful upload to s3 bucket, the update stac catalog service is called to update the catalog and to transfer the file from the temp bucket to the "rs-cluster-catalog" bucket.

In [18]:
# Clean previous executions
for collection in ("esmeralda:S1_L1", "quasimodo:S1_L1"):
    requests.delete(f"{url_catalog}/catalog/collections/DemoUser:s1_aux", **HEADERS)

In [19]:
# Create the user's collection first (this has to be done on client side)

from dataclasses import dataclass
import requests

@dataclass
class Collection:
    """A collection for test purpose."""

    user: str
    name: str

    @property
    def id_(self) -> str:
        """Returns the id."""
        return f"{self.user}_{self.name}"

    @property
    def properties(self):
        """Returns the properties."""
        return {
            "id": self.name,
            "type": "Collection",
            "links": [
                {
                    "rel": "items",
                    "type": "application/geo+json",
                    "href": f"http://localhost:8082/collections/{self.name}/items",
                },
                {"rel": "parent", "type": "application/json", "href": "http://localhost:8082/"},
                {"rel": "root", "type": "application/json", "href": "http://localhost:8082/"},
                {
                    "rel": "self",
                    "type": "application/json",
                    "href": f"""http://localhost:8082/collections/{self.name}""",
                },
                {
                    "rel": "license",
                    "href": "https://creativecommons.org/licenses/publicdomain/",
                    "title": "public domain",
                },
            ],
            "extent": {
                "spatial": {"bbox": [[-94.6911621, 37.0332547, -94.402771, 37.1077651]]},
                "temporal": {"interval": [["2000-02-01T00:00:00Z", "2000-02-12T00:00:00Z"]]},
            },
            "license": "public-domain",
            "description": "Some description",
            "stac_version": "1.0.0",
            "owner": user,
        }
    
user = "DemoUser"
mission = "s1"

# Create the collection for DemoUser
collection_type = Collection(user, f"{mission}_aux")
response = requests.post(url_catalog + f"/catalog/collections", json=collection_type.properties, **HEADERS)
response.raise_for_status()

collection = json.loads(response.content)
collection

{'id': 's1_aux',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser/collections/s1_aux/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://dev-rspy.esa-copernicus.eu/catalog/DemoUser/collections/s1_aux'},
  {'rel': 'items',
   'type': 'application/geo+json',
   'href': 'http://localhost:8082/collections/s1_aux/items'},
  {'rel': 'license',
   'href': 'https://creativecommons.org/licenses/publicdomain/',
   'title': 'public domain'}],
 'extent': {'spatial': {'bbox': [[-94.6911621,
     37.0332547,
     -94.402771,
     37.1077651]]},
  'temporal': {'interval': [['2000-02-01T00:00:00Z',
     '2000-02-12T00:00:00Z']]}},
 'license': 'pub

In [20]:


from datetime import datetime

from rs_workflows.common import (
    PrefectFlowConfig,
    download_flow,
)
    
def run_flow(user, url, url_catalog, station, mission, tmp_local_download, bucket_url, no_of_tasks, start_date, stop_date):
    # start the prefect flow
    download_flow(PrefectFlowConfig(user,
                                    url,
                                    url_catalog,
                                    station,
                                    mission,
                                    tmp_local_download,
                                    bucket_url,
                                    no_of_tasks,
                                    datetime.strptime(start_date, "%Y-%m-%dT%H:%M:%SZ"),
                                    datetime.strptime(stop_date, "%Y-%m-%dT%H:%M:%SZ"),                                   
            )
)


stations = ["CADIP", "ADGS"]
url = "http://rs-server-{}:8000"

tmp_local_download = "/tmp/{}_tmp"
no_of_tasks = 1

for station in stations:
    run_flow(user,
             url.format(station.lower()),
             url_catalog,
             station,
             mission,
             tmp_local_download.format(station),
             bucket_url + f"/{station}",
             no_of_tasks,
             "2014-01-01T12:00:00Z",
             "2024-02-20T12:00:00Z",
             )    
    

15:56:41.722 | INFO    | prefect.engine - Created flow run 'awesome-hippo' for flow 'download-flow'

15:56:41.724 | INFO    | prefect.task_runner.dask - Creating a new Dask cluster with `distributed.deploy.local.LocalCluster`

15:56:41.839 | INFO    | distributed.http.proxy - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy

15:56:41.841 | INFO    | distributed.scheduler - State start

15:56:41.846 | INFO    | distributed.scheduler -   Scheduler at:     tcp://127.0.0.1:35791

...

15:56:43.741 | INFO    | distributed.scheduler - Remove worker <WorkerState 'tcp://127.0.0.1:38511', name: 3, status: closing, memory: 0, processing: 0> (stimulus_id='handle-worker-cleanup-1711724203.7411797')

15:56:43.742 | INFO    | distributed.scheduler - Lost all workers

15:56:43.851 | INFO    | distributed.scheduler - Scheduler closing due to unknown reason...

15:56:43.853 | INFO    | distributed.scheduler - Scheduler closing all comms

15:56:43.855 | INFO    | Flow run 'mini-wasp' - Finished in state Completed()

In [21]:
import json
catalog_data = json.loads((requests.get(url_catalog + f"/catalog/collections/{user}:{mission}_aux/items").content.decode()))

for feature in catalog_data['features']:
    print(requests.get(url_catalog + f"/catalog/collections/{user}:{mission}_aux/items/{feature['id']}/download/file").content)

KeyError: 'features'